In [1]:
import pandas  as pd

dataset = pd.read_csv('../ozhegov_dataset.csv')
dataset = dataset[dataset['title'].str.len() > 0]
dataset = dataset[dataset['text'].str.len() > 0]
#для lenta - подсовываем даты
#dataset['title'] = dataset['date']
dataset = dataset.sample(n=1000)

dataset.astype({"text": str, "title": str})
dataset.info(show_counts=True)

dataset.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 25924 to 27624
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   title       1000 non-null   object
 2   text        1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


,Unnamed: 0,title,text
25924,25924,СОПЛИ,"Слизь, вытекающая из носа. * На соплях (держит..."
4900,4900,ГРАФА,"Полоса или столбец на бумажном листе, ограниче..."
15268,15268,ОБРЫВОЧНЫЙ,"О словах, мыслях, каких-н. сведениях: несвязны..."
19955,19955,ПОТОМУ,"По той причине, вследствие чего-н. Почему ты с..."
16816,16816,ОТЧАСТИ,"Не вполне, в нек-рой степени, частично. Прав т..."


In [2]:
%pip install -U sentence-transformers ipywidgets weaviate-client chardet charset-normalizer

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.0 which is incompatible.
opentelemetry-proto 1.23.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.0 which is incompatible.



   ---------------------------------------- 0.0/324.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/324.9 kB ? eta -:--:--
   ------- ------------------------------- 61.4/324.9 kB 656.4 kB/s eta 0:00:01
   --------------------- ------------------ 174.1/324.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 324.9/324.9 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/426.9 kB ? eta -:--:--
   ---------------------------------------- 426.9/426.9 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
  Attempting uninstall: weaviate-client
    Found existing installation: weaviate-client 4.5.7
    Uninstalling weaviate-client-4.5.7:
      Successfully uninstalled weaviate-client-4.5.7


In [3]:
from weaviate.classes.config import VectorDistances
models = [
    "intfloat/multilingual-e5-large",
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli",
    "cointegrated/LaBSE-en-ru",
    "sentence-transformers/LaBSE"
]

distances = [
    VectorDistances.L2_SQUARED,
    VectorDistances.DOT,
    VectorDistances.COSINE
]

In [4]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout


client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051
)


In [20]:
from tqdm import tqdm 
from weaviate.classes.config import Configure, Property, DataType, Tokenization, VectorDistances

def create_collection(client, model_name, distance):
    
    client.collections.create(
        "title",
        vectorizer_config=[
            Configure.NamedVectors.text2vec_huggingface(
                name="title_vector",
                source_properties=["title"],
                model=model_name,
                vector_index_config=Configure.VectorIndex.hnsw(distance_metric=distance)
            )
        ]
    )

    collection = client.collections.get("title")
    
    with collection.batch.dynamic() as batch:
        for i, data in tqdm(dataset.iterrows()):
            obj = {
                "title": data["title"]
            }
            batch.add_object(
                properties = obj
            )

    if len(collection.batch.failed_objects) > 0:
         print(collection.batch.failed_objects)

def delete_collection(client):
    client.collections.delete("title")


In [21]:
delete_collection(client)
create_collection(client, "cointegrated/LaBSE-en-ru", VectorDistances.COSINE)

1000it [00:04, 204.12it/s]


[ErrorObject(message='vectorize target vector title_vector: update vector: failed with status: 429 error: Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate', object_=_BatchObject(collection='Title', vector=None, uuid='d9786a45-178d-4ff5-bf3b-a5c14547ff00', properties={'title': 'СОПЛИ'}, tenant=None, references=None, retry_count=0), original_uuid='d9786a45-178d-4ff5-bf3b-a5c14547ff00'), ErrorObject(message='vectorize target vector title_vector: update vector: failed with status: 429 error: Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate', object_=_BatchObject(collection='Title', vector=None, uuid='d963a390-6a62-497a-a0f1-5b556d41f9d1', properties={'title': 'ГРАФА'}, tenant=None, references=None, retry_count=0), original_uuid='d963a390-6a62-497a-a0f1-5b556d41f9d1'), ErrorObject(mes

In [19]:
collection = client.collections.get("title")
print(collection)
for item in collection.iterator():
    print(item)

<weaviate.Collection config={
  "name": "Title",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false,
    "auto_tenant_creation": false
  },
  "properties": [
    {
      "name": "title",
      "description": "This property was generated by Weaviate's auto-schema feature on Sun May 26 08:50:16 2024",
      "data_type": "text",
      "index_filterable": true,
      "index_searchable": true,
      "nested_properties": null,
      "tokenization": "word",
      "vectorizer_config": null,
      "vectorizer": "none"
    }
  ],
  "references": [],
  "replication_config": {
    "factor": 1
  },
  "reranker_config": null,
  "sharding_c